In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics

class NB_SVM(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import sklearn.datasets
import re
import numpy as np

In [3]:
def clearstring(string):
    string = re.sub('[^\'\"A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string

def separate_dataset(trainset):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

In [4]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
416809
416809


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

cv = StratifiedKFold(n_splits=10, shuffle=True)
X_raw, y_raw = np.array(trainset.data), np.array(trainset.target)

aucs = []
for train, test in cv.split(X_raw, y_raw):
    y_train = y_raw[train]
    X_train = X_raw[train]

    y_test = y_raw[test]
    X_test = X_raw[test]
    
    tfidf = TfidfVectorizer().fit(X_train)
    X_train_tr = tfidf.transform(X_train)
    X_test_tr = tfidf.transform(X_test)
    
    model = NB_SVM(C=1, dual=True, n_jobs=-1).fit(X_train_tr, y_train)
    y_preds = model.predict(X_test_tr)
    accuracy = accuracy_score(y_test, y_preds)
    aucs.append(accuracy)
    print(accuracy)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


0.8491267632664812
0.8479992323193551
0.8436735281416439
0.8561009548486157
0.8522828147117392
0.8478203497996689
0.8455171421031166
0.8435865444599069
0.8466097221555737
0.8471855655261769


In [5]:
# bag-of-word
bow = CountVectorizer().fit_transform(trainset.data)

#tf-idf, must get from BOW first
tfidf = TfidfTransformer().fit_transform(bow)

#hashing, default n_features, probability cannot divide by negative
hashing = HashingVectorizer(non_negative = True).fit_transform(trainset.data)

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(bow, trainset.target, test_size = 0.2)

model = NB_SVM(C=1, dual=True, n_jobs=-1).fit(train_X, train_Y)

predicted = model.predict(test_X)

print('accuracy validation set: ', np.mean(predicted == test_Y))

# print scores
print(metrics.classification_report(test_Y, predicted, target_names = trainset.target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


accuracy validation set:  0.88038914613373
             precision    recall  f1-score   support

      anger       0.89      0.90      0.90     11438
       fear       0.84      0.84      0.84      9545
        joy       0.89      0.92      0.91     28253
       love       0.79      0.72      0.75      6951
    sadness       0.92      0.91      0.92     24218
   surprise       0.71      0.71      0.71      2957

avg / total       0.88      0.88      0.88     83362



In [8]:
train_X, test_X, train_Y, test_Y = train_test_split(tfidf, trainset.target, test_size = 0.2)

model = NB_SVM(C=1, dual=True, n_jobs=-1).fit(train_X, train_Y)

predicted = model.predict(test_X)

print('accuracy validation set: ', np.mean(predicted == test_Y))

# print scores
print(metrics.classification_report(test_Y, predicted, target_names = trainset.target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


accuracy validation set:  0.8463568532424846
             precision    recall  f1-score   support

      anger       0.91      0.89      0.90     11364
       fear       0.86      0.82      0.84      9455
        joy       0.80      0.94      0.86     28262
       love       0.87      0.47      0.61      6880
    sadness       0.89      0.86      0.88     24376
   surprise       0.72      0.59      0.65      3025

avg / total       0.85      0.85      0.84     83362



In [9]:
train_X, test_X, train_Y, test_Y = train_test_split(hashing, trainset.target, test_size = 0.2)

model = NB_SVM(C=1, dual=True, n_jobs=-1).fit(train_X, train_Y)

predicted = model.predict(test_X)

print('accuracy validation set: ', np.mean(predicted == test_Y))

# print scores
print(metrics.classification_report(test_Y, predicted, target_names = trainset.target_names))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


accuracy validation set:  0.8227009908591445
             precision    recall  f1-score   support

      anger       0.90      0.89      0.90     11454
       fear       0.87      0.83      0.85      9515
        joy       0.76      0.94      0.84     28277
       love       0.88      0.38      0.53      6949
    sadness       0.87      0.83      0.84     24221
   surprise       0.70      0.47      0.56      2946

avg / total       0.83      0.82      0.81     83362

